In [1]:
import pandas as pd
import os
from collections import Counter
import shutil

In [3]:
def Search_Location(search_contig,Location,readFiles):
    global MGE_ARG_Determin
    MGE_ARG_Determin=0

    file=readFiles
    file=file[file['Contig']==search_contig]    
    file_MGE_location=file['Location'].to_list()

    for temp_location in file_MGE_location:
        MGE_start=temp_location.split('-')[0]
        MGE_start=int(MGE_start)
        MGE_end=temp_location.split('-')[-1]
        MGE_end=int(MGE_end)

        ARG_start=Location.split('-')[0]  
        ARG_start=int(ARG_start)
        ARG_end=Location.split('-')[-1] 
        ARG_end=int(ARG_end)
        if MGE_start <= ARG_start and ARG_end <= MGE_end:
            MGE_ARG_Determin=1

In [ ]:
ARG_path='/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/China_Res/' ###Here please enter the output of ResFinder in Abricate
ARG_path_list=os.listdir(ARG_path)

All_Mo_Files_path='/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/All_Mo_Files/' ###MGE location file as modified in Step1
All_Mo_Files_list=os.listdir(All_Mo_Files_path)

ARG_Final_Motifi_path='/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/China_Res_on_MGE/'
###The modified Res file will be output under ARG_Final_Motifi_path, keeping only the MGE-related entries

China_436=pd.read_excel('china_436.xlsx')['Strain'].to_list()
print('China Strain is',len(China_436))

for i in China_436:
    # print(i)
    readFiles=[x for x in All_Mo_Files_list if x.startswith(i)]
    readFiles=[pd.read_csv(All_Mo_Files_path+file) for file in readFiles]
    readFiles=pd.concat(readFiles, ignore_index=True)

    ARG_File=[x for x in ARG_path_list if x.startswith(i)][0]
    ARG_motify=pd.read_csv(ARG_path+ARG_File,sep='\t')
    ARG_motify=ARG_motify[ARG_motify['GENE'] != "aac(6')-Iaa_1"].reset_index(drop=True)
    
    ARG_motify_contig=ARG_motify['SEQUENCE'].to_list()  ###
    ARG_motify_start=ARG_motify['START'].to_list()  #####
    ARG_motify_end=ARG_motify['END'].to_list()
    ARG_motify_name=ARG_motify['GENE'].to_list() ######


    counter=list(range(len(ARG_motify_contig)))
    final_count=[]
    for ii in counter:
        search_contig=ARG_motify_contig[ii]
        ARG_location=str(ARG_motify_start[ii])+'-'+str(ARG_motify_end[ii])
        Search_Location(search_contig,ARG_location,readFiles)
        final_count.append(MGE_ARG_Determin)

##### Now modify the ARG's File to output the ARGs that are on the MGE individually, deleting the ARGs that are not on the MGE #####
    Delet_row=[index for index, value in enumerate(final_count) if value == 0]
    # print(Delet_row)
    if len(Delet_row) >= 1:
        ARG_motify=ARG_motify.drop(Delet_row, axis=0).reset_index(drop=True)
    ARG_motify.to_csv(ARG_Final_Motifi_path+ARG_File+'.modified.csv',index=False)